In [13]:
# Shhhhhhh......
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [14]:
# Config Variables
# We'll do one at a time for now instead of just looping through mongoDB for all of them.
company = "GOOG"

# What is the size of the rolling window that decides what 0 and 1 represents? 
days = 30

In [15]:
# Get data from mongoDB
import ezmongo
df = ezmongo.get_data(company,"train")

In [19]:
new_column_order = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open','Volume', 'BuyOrSell']
df=df.reindex(columns=new_column_order)

In [21]:
# We're going to set the index to date because the .rolling will try to roll over that too!
# And bug out because you can't apply a rolling window over dates
# Also yes, I'm recycling the variable name 'df' but it's to save on memory!
# No it's not. I'm just too lazy.
df = df.set_index('Date')

In [22]:
def scaling_func(x):
    me_max = max(x)
    me_min = min(x)
    
    # Worked with dan on this part. This scales it as needed 
    index_of_last_element = len(x)-1
    where_im_at = x[index_of_last_element]
    return (where_im_at - me_min) / (me_max - me_min)

rolled = df.rolling(days).apply(scaling_func)
rolled.dropna(inplace=True)
rolled.reset_index(inplace=True)

In [25]:
#Save to mongoDB

ezmongo.set_data(rolled, 'GOOG', 'final')

True